In [18]:
import chromadb

In [19]:
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(
    name="cv_doc",
    metadata={"hnsw:space": "cosine"}
)

In [21]:
collection.add(
    documents=["Encantado, mi nombre es Martín y soy un desarrollador de Software con 2 años de experiencia en el sector de la tecnología", 
               "Mis últimos proyectos han sido: desarrollo de un chatbot empleando inteligencia artificial, un proyecto sobre el teorema del mono infinito, una aplicación de ciudadanía móvil", 
               "Tengo estudios de desarrollador de aplicaciones multiplataforma, desarrollo y diseño web, gestión de ventas y marketing"],
    metadatas=[{"documment": "CV"}, {"document": "CV"}, {"document": "CV"}],
    ids=["id1", "id2", "id3"]
)

Add of existing embedding ID: id1
Add of existing embedding ID: id2
Add of existing embedding ID: id3
Insert of existing embedding ID: id1
Insert of existing embedding ID: id2
Insert of existing embedding ID: id3


In [22]:
print(collection)

Collection(name=cv_doc)


In [24]:
results = collection.query(
    query_texts=["marketing"],
    n_results=2
)

In [25]:
print(results)

{'ids': [['id3', 'id1']], 'embeddings': None, 'documents': [['Tengo estudios de desarrollador de aplicaciones multiplataforma, desarrollo y diseño web, gestión de ventas y marketing', 'Encantado, mi nombre es Martín y soy un desarrollador de Software con 2 años de experiencia en el sector de la tecnología']], 'uris': None, 'data': None, 'metadatas': [[{'document': 'CV'}, {'documment': 'CV'}]], 'distances': [[0.9601737260818481, 1.6389015913009644]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
